In [1]:
from __future__ import division, absolute_import, print_function
import os, sys, time, re, json
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
import StringIO
import json

imread = plt.imread
def imread8(im_file):
    ''' Read image as a 8-bit numpy array '''
    im = np.asarray(Image.open(im_file))
    return im

def read_png(res):
    img = Image.open(StringIO.StringIO(res))
    return np.array(img)

def read_npy(res):
    return np.load(res)

# Matches a color from the object_mask and then returns that region color
def match_color(object_mask, target_color, tolerance=3):
    match_region = np.ones(object_mask.shape[0:2], dtype=bool)
    for c in range(3): # r,g,b
        min_val = target_color[c] - tolerance
        max_val = target_color[c] + tolerance
        channel_region = (object_mask[:,:,c] >= min_val) & (object_mask[:,:,c] <= max_val)
        match_region &= channel_region

    if match_region.sum() != 0:
        return match_region
    else:
        return None
    
# Swap colors method
def swap_color(imgarray, source, dest):
    matched_color = match_color(imgarray, [source.R, source.G, source.B])
    imgarray[:,:,:3][matched_color] = [dest.R, dest.G, dest.B]
    return np.array(imgarray)


Connect to the game
===================
Load unrealcv python client, do :code:`pip install unrealcv` first.



In [4]:
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

INFO:__init__:204:Got connection confirm: 'connected to NeighborhoodCV'


In [5]:
# Make sure the connection works well

res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print(res)

Is Listening
Client Connected
9000
Configuration
Config file: C:/Program Files/Epic Games/UE_4.13/Engine/Binaries/Win64/unrealcv.ini
Port: 9000
Width: 1440
Height: 900



Get objects
======================
Write a json file with the object and their corresponding classes.



In [6]:
scene_objects = client.request('vget /objects').split(' ')
print('Number of objects in this scene:', len(scene_objects))

if '257' in scene_objects:
    scene_objects.remove('257')

obj_id_to_class = {}
for obj_id in scene_objects:
    obj_id_parts = obj_id.split('_')
    class_name = obj_id_parts[0]    
    obj_id_to_class[obj_id] = class_name

# Write JSON file
with open('neighborhood_object_ids.json', 'w') as outfile:
    json.dump(obj_id_to_class, outfile)

Number of objects in this scene: 3762


Get object colors
======================
First we create the color object class

In [7]:
class Color(object):
    ''' A utility class to parse color value '''
    regexp = re.compile('\(R=(.*),G=(.*),B=(.*),A=(.*)\)')
    def __init__(self, color_str):
        self.color_str = color_str
        match = self.regexp.match(color_str)
        (self.R, self.G, self.B, self.A) = [int(match.group(i)) for i in range(1,5)]

    def __repr__(self):
        return self.color_str

Then, we either load from json

In [8]:
id2color = {}
with open('id2color.json') as data_file:
    data = json.load(data_file)

for obj_id in data.keys():
    color_map = data[obj_id]
    color_str = '(R=' + str(color_map['R']) + ',G=' + \
                str(color_map['G']) + ',B=' + str(color_map['B']) + \
                ',A=' + str(color_map['A']) + ')'
    color = Color(color_str)
    id2color[obj_id] = color

or We load from the scene

In [7]:
id2color = {} # Map from object id to the labeling color
for i, obj_id in enumerate(scene_objects):
    color = Color(client.request('vget /object/%s/color' % obj_id))
    id2color[obj_id] = color
    print('%d. %s : %s' % (i, obj_id, str(color)))

0. WorldSettings_1 : (R=0,G=0,B=0,A=255)
1. InstancedFoliageActor_0 : (R=0,G=0,B=255,A=255)
2. SkySphere : (R=0,G=255,B=0,A=255)
3. Landscape_0 : (R=0,G=255,B=255,A=255)
4. Road_2 : (R=255,G=0,B=0,A=255)
5. Road_3 : (R=255,G=0,B=255,A=255)
6. Road_5 : (R=255,G=255,B=0,A=255)
7. Road_7 : (R=255,G=255,B=255,A=255)
8. Road_9 : (R=0,G=0,B=127,A=255)
9. Road_10 : (R=0,G=255,B=127,A=255)
10. Road_11 : (R=255,G=0,B=127,A=255)
11. Road_12 : (R=255,G=255,B=127,A=255)
12. Road_13 : (R=0,G=127,B=0,A=255)
13. Road_14 : (R=0,G=127,B=255,A=255)
14. Road_15 : (R=255,G=127,B=0,A=255)
15. Road_17 : (R=255,G=127,B=255,A=255)
16. Road_18 : (R=0,G=127,B=127,A=255)
17. Road_19 : (R=255,G=127,B=127,A=255)
18. Road_20 : (R=127,G=0,B=0,A=255)
19. Road_21 : (R=127,G=0,B=255,A=255)
20. Road_22 : (R=127,G=255,B=0,A=255)
21. Road_23 : (R=127,G=255,B=255,A=255)
22. Road_24 : (R=127,G=0,B=127,A=255)
23. Road_25 : (R=127,G=255,B=127,A=255)
24. Road_26 : (R=127,G=127,B=0,A=255)
25. Road_27 : (R=127,G=127,B=255,A=255)

KeyboardInterrupt: 

Write to JSON if loaded from scene

In [ ]:
# Convert to serializable json dictionary
serializable_map = {}
for color_id in id2color.keys():
    curr_color = id2color[color_id]
    color_map = {}
    color_map['R'] = curr_color.R
    color_map['G'] = curr_color.G
    color_map['B'] = curr_color.B
    color_map['A'] = curr_color.A
    serializable_map[color_id] = color_map

# Write to JSON
with open('id2color.json', 'w') as outfile:
    json.dump(serializable_map, outfile)

In [9]:
# Map classes to lists of objects
classes = {}

for obj_id in obj_id_to_class.keys():
    
    curr_class = obj_id_to_class[obj_id]
    if curr_class not in classes:
        classes[curr_class] = []
    
    classes[curr_class].append(obj_id)

# Write classes to json
with open('neighborhood_classes.json', 'w') as outfile:
    json.dump(classes, outfile) 

In [18]:
# Get top 20 classes from json file
with open('class2count.json') as data_file:
    class2count = json.load(data_file)

from operator import itemgetter
class2countlist = sorted(class2count.items(), key=itemgetter(1), reverse=True)

top20 = []
top20.append('dummy')
for cls in class2countlist[0:21]:
    top20.append(cls[0])

In [ ]:
# Normalize using built in API
counter = 0
for curr_class in classes.keys():
    
    
    object_list = classes[curr_class]
    curr_color = id2color[object_list[0]]
    
    for obj_id in object_list:
        
        if curr_class in top20:
            client.request('vset /object/' + obj_id + '/color ' + \
                       str(top20.index(curr_class)) + ' 0 0')
        
            print(str(counter) + '. vset /object/' + obj_id + '/color ' + \
                       str(top20.index(curr_class)) + ' 0 0')
        else:
            client.request('vset /object/' + obj_id + '/color 0 0 0')
            print(str(counter) + '. vset /object/' + obj_id + '/color 0 0 0')
    
        counter += 1

0. vset /object/Fir_01_476/color 5 0 0
1. vset /object/Fir_19/color 5 0 0
2. vset /object/Fir_18/color 5 0 0
3. vset /object/Fir_17/color 5 0 0
4. vset /object/Fir_16/color 5 0 0
5. vset /object/Fir_15/color 5 0 0
6. vset /object/Fir_12/color 5 0 0
7. vset /object/Fir_11/color 5 0 0
8. vset /object/Fir_10/color 5 0 0
9. vset /object/Fir_139/color 5 0 0
10. vset /object/Fir_482/color 5 0 0
11. vset /object/Fir_226/color 5 0 0
12. vset /object/Fir_44/color 5 0 0
13. vset /object/Fir_45/color 5 0 0
14. vset /object/Fir_46/color 5 0 0
15. vset /object/Fir_47/color 5 0 0
16. vset /object/Fir_40/color 5 0 0
17. vset /object/Fir_41/color 5 0 0
18. vset /object/Fir_42/color 5 0 0
19. vset /object/Fir_43/color 5 0 0
20. vset /object/Fir_492/color 5 0 0
21. vset /object/Fir_223/color 5 0 0
22. vset /object/Fir_229/color 5 0 0
23. vset /object/Fir_183/color 5 0 0
24. vset /object/Fir_235/color 5 0 0
25. vset /object/Fir_232/color 5 0 0
26. vset /object/Fir_95/color 5 0 0
27. vset /object/Fir_93/c

173. vset /object/Inner_Door_Swing_01_C_40/color 0 0 0
174. vset /object/Inner_Wall_8th_01_163/color 0 0 0
175. vset /object/Inner_Wall_Close_Gap_233/color 0 0 0
176. vset /object/Inner_Wall_Close_Gap_235/color 0 0 0
177. vset /object/Inner_Wall_Close_Gap_239/color 0 0 0
178. vset /object/Inner_Wall_Quart_Arch_157/color 0 0 0
179. vset /object/Inner_Wall_Half_01_278/color 0 0 0
180. vset /object/Inner_Wall_Half_Door_24/color 0 0 0
181. vset /object/Inner_Wall_Half_Door_21/color 0 0 0
182. vset /object/Inner_Wall_Quart_Door_57/color 0 0 0
183. vset /object/Inner_Wall_Close_Gap_19/color 0 0 0
184. vset /object/Inner_Wall_Quart_Door_110/color 0 0 0
185. vset /object/Inner_Wall_Close_Gap_242/color 0 0 0
186. vset /object/Inner_Wall_Half_Door_39/color 0 0 0
187. vset /object/Inner_Wall_Half_Door_34/color 0 0 0
188. vset /object/Inner_Wall_Half_Door_36/color 0 0 0
189. vset /object/Inner_Wall_Quart_plus_8th_5/color 0 0 0
190. vset /object/Inner_Wall_Quart_plus_8th_6/color 0 0 0
191. vset /ob

351. vset /object/Hedge_275/color 1 0 0
352. vset /object/Hedge_133/color 1 0 0
353. vset /object/Hedge_179/color 1 0 0
354. vset /object/Hedge_178/color 1 0 0
355. vset /object/Hedge_177/color 1 0 0
356. vset /object/Hedge_176/color 1 0 0
357. vset /object/Hedge_174/color 1 0 0
358. vset /object/Hedge_173/color 1 0 0
359. vset /object/Hedge_172/color 1 0 0
360. vset /object/Hedge_171/color 1 0 0
361. vset /object/Hedge_170/color 1 0 0
362. vset /object/Hedge_549/color 1 0 0
363. vset /object/Hedge_546/color 1 0 0
364. vset /object/Hedge_540/color 1 0 0
365. vset /object/Hedge_541/color 1 0 0
366. vset /object/Hedge_389/color 1 0 0
367. vset /object/Hedge_384/color 1 0 0
368. vset /object/Hedge_385/color 1 0 0
369. vset /object/Hedge_386/color 1 0 0
370. vset /object/Hedge_380/color 1 0 0
371. vset /object/Hedge_381/color 1 0 0
372. vset /object/Hedge_382/color 1 0 0
373. vset /object/Hedge_536/color 1 0 0
374. vset /object/Hedge_13/color 1 0 0
375. vset /object/Hedge_534/color 1 0 0
3

556. vset /object/Hedge_312/color 1 0 0
557. vset /object/Hedge_310/color 1 0 0
558. vset /object/Hedge_831/color 1 0 0
559. vset /object/Hedge_314/color 1 0 0
560. vset /object/Hedge_319/color 1 0 0
561. vset /object/Hedge_318/color 1 0 0
562. vset /object/Hedge_406/color 1 0 0
563. vset /object/Hedge_420/color 1 0 0
564. vset /object/Hedge_428/color 1 0 0
565. vset /object/Hedge_128/color 1 0 0
566. vset /object/Hedge_129/color 1 0 0
567. vset /object/Hedge_124/color 1 0 0
568. vset /object/Hedge_126/color 1 0 0
569. vset /object/Hedge_127/color 1 0 0
570. vset /object/Hedge_120/color 1 0 0
571. vset /object/Hedge_121/color 1 0 0
572. vset /object/Hedge_122/color 1 0 0
573. vset /object/Hedge_123/color 1 0 0
574. vset /object/Hedge_655/color 1 0 0
575. vset /object/Hedge_391/color 1 0 0
576. vset /object/Hedge_106/color 1 0 0
577. vset /object/Hedge_339/color 1 0 0
578. vset /object/Hedge_335/color 1 0 0
579. vset /object/Hedge_334/color 1 0 0
580. vset /object/Hedge_337/color 1 0 0


762. vset /object/Hedge_201/color 1 0 0
763. vset /object/Hedge_202/color 1 0 0
764. vset /object/Hedge_464/color 1 0 0
765. vset /object/Hedge_467/color 1 0 0
766. vset /object/Hedge_466/color 1 0 0
767. vset /object/Hedge_51/color 1 0 0
768. vset /object/Hedge_50/color 1 0 0
769. vset /object/Hedge_53/color 1 0 0
770. vset /object/Hedge_52/color 1 0 0
771. vset /object/Hedge_55/color 1 0 0
772. vset /object/Hedge_57/color 1 0 0
773. vset /object/Hedge_56/color 1 0 0
774. vset /object/Hedge_59/color 1 0 0
775. vset /object/Hedge_58/color 1 0 0
776. vset /object/Hedge_159/color 1 0 0
777. vset /object/Hedge_511/color 1 0 0
778. vset /object/Hedge_510/color 1 0 0
779. vset /object/Hedge_512/color 1 0 0
780. vset /object/Hedge_515/color 1 0 0
781. vset /object/Hedge_514/color 1 0 0
782. vset /object/Hedge_517/color 1 0 0
783. vset /object/Hedge_516/color 1 0 0
784. vset /object/Hedge_519/color 1 0 0
785. vset /object/Hedge_518/color 1 0 0
786. vset /object/Hedge_425/color 1 0 0
787. vset

...

1146. vset /object/Stairs_Prefab_149/color 0 0 0
1147. vset /object/Stairs_Beam_Quart_01_50/color 0 0 0
1148. vset /object/Stairs_Beam_Quart_100/color 0 0 0
1149. vset /object/Stairs_Prefab_1341/color 0 0 0
1150. vset /object/Stairs_Beam_Quart_8/color 0 0 0
1151. vset /object/Stairs_Beam_Quart_9/color 0 0 0
1152. vset /object/Stairs_Beam_Quart_2/color 0 0 0
1153. vset /object/Stairs_Beam_Quart_4/color 0 0 0
1154. vset /object/Stairs_Beam_Quart_5/color 0 0 0
1155. vset /object/Stairs_Beam_Quart_51/color 0 0 0
1156. vset /object/Stairs_Prefab_429/color 0 0 0
1157. vset /object/Stairs_Beam_Quart_72/color 0 0 0
1158. vset /object/Laptop_01_51/color 0 0 0
1159. vset /object/Oven_152/color 0 0 0
1160. vset /object/Oven_01_66/color 0 0 0
1161. vset /object/Oven_212/color 0 0 0
1162. vset /object/Oven_195/color 0 0 0
1163. vset /object/sofa_long_01_99/color 0 0 0
1164. vset /object/sofa_long_2/color 0 0 0
1165. vset /object/sofa_long_3/color 0 0 0
1166. vset /object/small_table_03_87/color 0 0

1314. vset /object/Kitchen_Unit_16/color 0 0 0
1315. vset /object/Kitchen_Unit_17/color 0 0 0
1316. vset /object/Kitchen_Unit_10/color 0 0 0
1317. vset /object/Kitchen_Unit_12/color 0 0 0
1318. vset /object/Kitchen_Unit_13/color 0 0 0
1319. vset /object/Kitchen_Unit_18/color 0 0 0
1320. vset /object/Kitchen_Splash_Tile_15/color 0 0 0
1321. vset /object/Kitchen_Unit_27/color 0 0 0
1322. vset /object/Kitchen_Splash_Tile_36/color 0 0 0
1323. vset /object/Kitchen_Cupboard_146/color 0 0 0
1324. vset /object/Kitchen_Splash_Tile_215/color 0 0 0
1325. vset /object/Kitchen_Unit_Corner_01_180/color 0 0 0
1326. vset /object/Kitchen_Cupboard_02_22/color 0 0 0
1327. vset /object/Kitchen_Unit_21/color 0 0 0
1328. vset /object/Kitchen_Cupboard_Fan_01_196/color 0 0 0
1329. vset /object/Kitchen_Unit_02_11/color 0 0 0
1330. vset /object/Kitchen_Unit_56/color 0 0 0
1331. vset /object/Kitchen_Unit_2/color 0 0 0
1332. vset /object/Kitchen_Unit_3/color 0 0 0
1333. vset /object/Kitchen_Cupboard_Fan_209/color

1498. vset /object/Driveway_Edge_21/color 7 0 0
1499. vset /object/Driveway_Edge_22/color 7 0 0
1500. vset /object/Driveway_Edge_23/color 7 0 0
1501. vset /object/Driveway_Half_43/color 7 0 0
1502. vset /object/Driveway_96/color 7 0 0
1503. vset /object/Driveway_8th_17/color 7 0 0
1504. vset /object/Driveway_8th_16/color 7 0 0
1505. vset /object/Driveway_8th_15/color 7 0 0
1506. vset /object/Driveway_8th_14/color 7 0 0
1507. vset /object/Driveway_8th_13/color 7 0 0
1508. vset /object/Driveway_8th_12/color 7 0 0
1509. vset /object/Driveway_8th_11/color 7 0 0
1510. vset /object/Driveway_8th_10/color 7 0 0
1511. vset /object/Driveway_8th_19/color 7 0 0
1512. vset /object/Driveway_8th_18/color 7 0 0
1513. vset /object/Driveway_162/color 7 0 0
1514. vset /object/Driveway_Half_29/color 7 0 0
1515. vset /object/Driveway_38/color 7 0 0
1516. vset /object/Driveway_39/color 7 0 0
1517. vset /object/Driveway_32/color 7 0 0
1518. vset /object/Driveway_33/color 7 0 0
1519. vset /object/Driveway_30/

1679. vset /object/Drinks_Can_181/color 0 0 0
1680. vset /object/Drinks_Can_178/color 0 0 0
1681. vset /object/Drinks_Can_175/color 0 0 0
1682. vset /object/Drinks_Can_172/color 0 0 0
1683. vset /object/Drinks_Can_718/color 0 0 0
1684. vset /object/Drinks_Can_793/color 0 0 0
1685. vset /object/Drinks_Can_796/color 0 0 0
1686. vset /object/Drinks_Can_799/color 0 0 0
1687. vset /object/Drinks_Can_03_45/color 0 0 0
1688. vset /object/Drinks_Can_11/color 0 0 0
1689. vset /object/Drinks_Can_31/color 0 0 0
1690. vset /object/Drinks_Can_33/color 0 0 0
1691. vset /object/Drinks_Can_802/color 0 0 0
1692. vset /object/Drinks_Can_05_68/color 0 0 0
1693. vset /object/Drinks_Can_04_71/color 0 0 0
1694. vset /object/Drinks_Can_13/color 0 0 0
1695. vset /object/Drinks_Can_10/color 0 0 0
1696. vset /object/Drinks_Can_14/color 0 0 0
1697. vset /object/Drinks_Can_745/color 0 0 0
1698. vset /object/Drinks_Can_8/color 0 0 0
1699. vset /object/Drinks_Can_75/color 0 0 0
1700. vset /object/Drinks_Can_74/colo

1836. vset /object/Outer_Wall_Quart_Floor_30/color 3 0 0
1837. vset /object/Outer_Wall_Quart_Floor_31/color 3 0 0
1838. vset /object/Outer_Door_Swing_686/color 3 0 0
1839. vset /object/Outer_Wall_Half_18/color 3 0 0
1840. vset /object/Outer_Wall_Half_12/color 3 0 0
1841. vset /object/Outer_Wall_Half_13/color 3 0 0
1842. vset /object/Outer_Wall_Half_10/color 3 0 0
1843. vset /object/Outer_Wall_Half_16/color 3 0 0
1844. vset /object/Outer_Wall_Half_17/color 3 0 0
1845. vset /object/Outer_Wall_Half_15/color 3 0 0
1846. vset /object/Outer_Door_Swing_3723/color 3 0 0
1847. vset /object/Outer_Wall_16th_01_68/color 3 0 0
1848. vset /object/Outer_Wall_Half_Door_104/color 3 0 0
1849. vset /object/Outer_Wall_Half_Win_150/color 3 0 0
1850. vset /object/Outer_Wall_Half_Win_79/color 3 0 0
1851. vset /object/Outer_Wall_Half_Win_78/color 3 0 0
1852. vset /object/Outer_Wall_Half_Win_75/color 3 0 0
1853. vset /object/Outer_Wall_Half_Win_74/color 3 0 0
1854. vset /object/Outer_Wall_Half_Win_76/color 3 0

Begin Data Collection (Without Normalization)
======

In [ ]:
class2count = {}
for batch in range(1,101):

    # Get random location
    z = 300
    x = random.randint(-5500, 5500)
    y = random.randint(-5500, 5500)

    # Coordinates x, y, z
    client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                   ' ' + str(z)) 

    # Get 10 shots in a series
    angles = []
    a = 0
    while len(angles) < 20:
        angles.append(a)
        a -= 3

    # Increment height sequentially
    heights = []
    height = 300
    while len(heights) < 20:
        heights.append(height)
        height += 50

    for i,angle in enumerate(angles[1:2]):
        
        print("Batch: " + str(batch) + " , Image: " + str(i))
        
        # x, y, z
        client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                       ' ' + str(heights[i])) 

        # Pitch, yaw, roll
        client.request('vset /camera/0/rotation ' + str(angle) + ' 0 0')


        # Get Ground Truth
        res = client.request('vget /camera/0/object_mask png')
        object_mask = read_png(res)
        
        # Get all classes in image
        print("Getting all object_ids in image....")
        id2mask = {}
        for obj_id in scene_objects:
            color = id2color[obj_id]
            mask = match_color(object_mask, [color.R, color.G, color.B], tolerance = 3)
            if mask is not None:
                id2mask[obj_id] = mask
        
        # Update class frequency map
        for idmask in id2mask.keys():
            curr_class = obj_id_to_class[idmask]
            if curr_class not in class2count:
                class2count[curr_class] = 1
            else: 
                class2count[curr_class] += 1
               
        # Write to file
        normalized_img = Image.fromarray(object_mask)
        grayscale_img = normalized_img.convert('L')

        directory = './batch/round' + str(batch) + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        grayscale_img.save('./batch/round' + str(batch) + '/seg' + \
                            str(i) + '.png')
        
        
        res = client.request('vget /camera/0/lit png')
        normal = read_png(res)
        normal = Image.fromarray(normal)
        
        normal.save('./batch/round' + str(batch) + '/pic' + \
                            str(i) + '.png')
        
        print("Images written. ")

with open('class2count.json', 'w') as outfile:
    json.dump(class2count, outfile)

Batch: 1 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 2 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 3 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 4 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 5 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 6 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 7 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 8 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 9 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 10 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written. 
Batch: 11 , Image: 0
Getting all object_ids in image....
(900L, 1440L, 4L)
Images written